In [ ]:
import cv2
from google.colab.patches import cv2_imshow # colab does not support cv2.imshow()

# Download & compile the YOLO Darknet network into Google Colab



1. Link you Google drive account to colab
2. Create a darknet folder (in my case : /content/drive/MyDrive/Colab Notebooks/YOLO/darknet)
3. Jump to this directory %cd "/content/drive/MyDrive/Colab Notebooks/YOLO"



In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/YOLO"

/content/drive/MyDrive/Colab Notebooks/YOLO


5. Download darknet from https://github.com/pjreddie/darknet  

This command will create a new ./darknet folder with all the darket sources & config files (do that once !)

In [ ]:
#!git clone https://github.com/pjreddie/darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (10399/10399), done.
Checking out files: 100% (2044/2044), done.


6. Open the makefile file and change these line  
GPU=1  
CUDNN=1  
OPENCV=1  

7. Now we need to compile the network in colab

In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/YOLO'

In [ ]:
%cd ./darknet

/content/drive/My Drive/Colab Notebooks/YOLO/darknet


In [ ]:
!chmod +rwx ./*

Build the netword (do that once!)

In [ ]:
!make

7. Copy the YOLO v4 config & weights file into toe /cfg folder

8. Download the pre-trained weights for the training (https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137)

# Build the Network Config file

1. Copy the yolov4.cfg file an create this new copy: document_yolov4.cfg
2. Open and modify the file with these entries  
Line 3 : subdivisions=12  
Line 7: width=416  
Line 8: height=416  
Line 19: max_batches = 2000  
Line 21: steps=1600,1800  
Line 961,1049,1137: filters=18 # (classes+5)*3  
Line 968,1056,1144: classes=1



# Test the YOLO network

In [ ]:
!./darknet detector test cfg/coco.data "/content/drive/MyDrive/Colab Notebooks/YOLO/oc_data/yolov4.cfg"  "/content/drive/MyDrive/Colab Notebooks/YOLO/oc_data/yolov4.weights"  "/content/drive/MyDrive/Colab Notebooks/YOLO/yoloimg.jpg" 

# Prepare the custom model

1. Get images from any sources ...
2. Label images by using for example LabelImg (https://github.com/tzutalin/labelImg)

After labeling all your images (here i'll label only 300 images) you should have 3 kind of files:
* the images itself (jpg, png, etc.)
* a set of text file (one per image) associated with the coordinates of the boxes in the image (thes files are generated by LabelImg)
* a classes file which list the classes

Here the images are located in /content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/doc_images

# Configuration files

## Split the dataset (Train/Test)

1. Create one file with the list of 80% of the images (train) : 240 images in our case (document_training.txt)
2. Create one file with the list of 20% of the images (test) : 60 images in our case (document_test.txt)
3. Create a folder named doc_data into the root of darknet (/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data)
4. Put the two files in this directory

*Note: the list of image file must repect the correct path (starting into the darknet folder)*

In [ ]:
from os import listdir
from os.path import isfile, join
mydirectory = "/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/doc_images"
allfiles = [f for f in listdir(mydirectory) if isfile(join(mydirectory, f))]

List all files in the directory (images + labels + classes)

In [ ]:
len(allfiles)

589

Create the two files (train + test) by splitting per 80%-20% (do that once !)

In [ ]:
rootdir = "doc_data/doc_images/"
counter = 1
images_files = []

# put all the images files into a list
for file in allfiles:
  if (file[len(file)-3:len(file)] != "txt"):
    images_files.append(rootdir +  file)

# Set the size of the test (20%) & train data (80%)
training_limit = int(len(images_files) * 80 / 100)
test_limit = len(images_files) - training_limit

# Build the two lists
images_training_files = []
images_test_files = []
for file in images_files:
  if (counter <= training_limit):
    images_training_files.append(file)
  else:
    images_test_files.append( file)
  counter = counter + 1

# Put the two list into two files
with open('/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/document_test.txt', 'w') as f:
    for item in images_test_files:
        f.write("%s\n" % item)

with open('/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/document_training.txt', 'w') as f:
    for item in images_training_files:
        f.write("%s\n" % item)

## List of labels/classes

7. Then create a file named **document.names** and put the label (only one here) in there.

## Configuration bundle file

Create a file named **document.data** which will regroup all the configuration. Put that content inside :

classes = 1  
valid = doc_data/document_test.txt  
train = doc_data/document_training.txt  
names = doc_data/document.names  
backup = backup  

# Train the model

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/YOLO/darknet"
!chmod +rwx ./*

/content/drive/MyDrive/Colab Notebooks/YOLO/darknet


**In case of problem ... just launch this command line below to restart from the last backup (not from the beggining).**

In [ ]:
!./darknet detector train doc_data/document.data doc_data/document_yolov4.cfg doc_data/yolov4.conv.137 -dont_show -map

In [ ]:
!./darknet detector train doc_data/document.data doc_data/document_yolov4.cfg backup/document_yolov4_last.weights -dont_show -map

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.698764), count: 10, class_loss = 1.137769, iou_loss = 0.163209, total_loss = 1.300978 
 total_bbox = 3043, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000009, iou_loss = 0.000000, total_loss = 0.000009 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.706794), count: 1, class_loss = 0.215954, iou_loss = 1.297284, total_loss = 1.513238 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.771787), count: 11, class_loss = 1.300854, iou_loss = 0.203669, total_loss = 1.504523 
 total_bbox = 3055, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000005, iou_loss = 0.000000, 